In [ ]:
f_func <- "./functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))

In [ ]:
DIR_IN_HUMAN="../../output/cfspi/samples/results/host_TLEN_readCount/"
samples_metadata <- read.csv(file = "../../output/samples_metadata.csv") %>% as.data.table()

list.files(DIR_IN_HUMAN, pattern = "_MT_TLEN_EndMotif.txt") %>% length()

In [ ]:
if (exists("host_MT_IS_meta")){rm("host_MT_IS_meta")}
profvis({
for (file in list.files(DIR_IN_HUMAN, pattern = "[pos/neg]_MT_TLEN_EndMotif.txt")){
    sample_tmp <- unlist(strsplit(file, "_"))[1] 
    side_tmp <- unlist(strsplit(file, "_"))[2]
    
    if(countLines(paste0(DIR_IN_HUMAN, file)) > 1){
        pre_tmp_IS <- fread(paste0(DIR_IN_HUMAN, file), sep = ",") %>% 
                    dplyr::rename(EndMotif_tmp = EndMotif) %>% 
                    mutate(sample_id = sample_tmp) %>% 
                    mutate(side = side_tmp) %>% 
                    mutate(read = substr(side_tmp, 0, 2)) %>% 
                    filter(TLEN != 0) %>% #remove TLEN not equal to 0
                    filter(abs(TLEN) < 1000) %>% #filter max TLEN of 1000
                    filter(!grepl(EndMotif_tmp, pattern = "N")) %>% #remove reads with end-motif including "N"
                    group_by(TLEN, EndMotif_tmp, sample_id, side) %>% 
                    summarise(Count = sum(Count), .groups = "keep")
        
        if (grepl(side_tmp, pattern = "neg")){
            tmp_IS <- pre_tmp_IS #%>% select(-any_of("EndMotif_tmp"))
            tmp_IS$EndMotif <- as.character(reverseComplement(DNAStringSet(pre_tmp_IS$EndMotif_tmp))) 
        } else {
            tmp_IS <- pre_tmp_IS #%>% select(-any_of("EndMotif_tmp"))
            tmp_IS$EndMotif <- pre_tmp_IS$EndMotif_tmp
        }
        
        tmp_IS_meta <- merge(x = samples_metadata, y = tmp_IS, by.x = "sample_id", by.y = "sample_id") 
        if (!exists("host_MT_IS_meta")){host_MT_IS_meta <- tmp_IS_meta} else {
            host_MT_IS_meta <- rbind(host_MT_IS_meta, tmp_IS_meta)}
    }
    fwrite(host_MT_IS_meta, "../../output/PP_figures/host_new_MT_IS_meta.csv", row.names = FALSE)
}
})